In [1]:
import numpy as np

In [ ]:
def mult_y(a: list, b: list, max_pow) -> list:
    ans = [0] * min((len(a) + len(b) - 1), max_pow + 1)
    for i in range(min(len(a), max_pow + 1)):
        # хотим вычислить коэффициент при y^((len(a) + len(b) - 1) - i)
        for j in range(min(max_pow - i + 1, len(b))):
            ans[len(ans) - i - j - 1] += a[len(a) - i - 1] * b[len(b) - j - 1]
    i = 0
    while i < len(ans) and ans[i] == 0:
        i += 1
    if i == len(ans):
        return [0]
    return ans[i:]

In [7]:
mult_y([5, 2, 7, 2], [3, 3, 3], 5)

[15, 21, 42, 33, 27, 6]

In [ ]:
assert mult_y([5, 2, 7, 2], [3, 3, 3], 2) == [33, 27, 6]

In [2]:
def show_local(arr: list, loc='y') -> list:
    step_y = len(arr) - 1
    ans = "("
    for j in range(len(arr) - 1):
        if arr[j] != 0:
            if arr[j] != 1:
                ans += str(arr[j]) + '*' + loc + '^' + str(step_y) + ' + '
            else:
                ans += loc + '^' + str(step_y) + ' + '
        step_y -= 1
    if arr[-1] != 0:
        ans += str(arr[-1])
    elif ans != '(' and ans[-2] == '+':
        ans = ans[:-3]
    return ans + ')'


def show_global(arr: list, glob='x', loc='y') -> list:
    step_x = len(arr) - 1
    ans = ""
    for i in range(len(arr) - 1):
        if arr[i] != [0] and show_local(arr[i], loc=loc) != '()':
            ans += show_local(arr[i], loc=loc) + '*' + glob +'^' + str(step_x) + ' + '
        step_x -= 1

    if list(arr[-1]) != [0] and show_local(arr[-1], loc=loc) != '()':
        ans += show_local(arr[-1], loc=loc)
    elif ans and ans[-2] == '+':
        ans = ans[:-2]
    return ans


def add_y(a: list, b: list) -> list:
    if len(a) < len(b):
        a, b = b, a
    ans = a[::]
    ind = len(ans) - 1
    for i in range(len(b) - 1, -1, -1):
        ans[ind] += b[i]
        ind -= 1
    i = 0
    while i < len(ans) and ans[i] == 0:
        i += 1
    if i == len(ans):
        return [0]
    return ans[i:]


def add(a: list, b: list) -> list:
    if len(a) < len(b):
        a, b = b, a
    ans = a[::]
    ind = len(ans) - 1
    for i in range(len(b) - 1, -1, -1):
        ans[ind] = add_y(ans[ind], b[i])
        ind -= 1
    return ans


def mult_y(a: list, b: list, max_pow) -> list:
    ans = [0] * min((len(a) + len(b) - 1), max_pow + 1)
    for i in range(min(len(a), max_pow + 1)):
        # хотим вычислить коэффициент при y^((len(a) + len(b) - 1) - i)
        for j in range(min(max_pow - i + 1, len(b))):
            ans[len(ans) - i - j - 1] += a[len(a) - i - 1] * b[len(b) - j - 1]
    i = 0
    while i < len(ans) and ans[i] == 0:
        i += 1
    if i == len(ans):
        return [0]
    return ans[i:]


def mult(a: list, b: list, x_size: int, y_pow: int) -> list:
    """
    вычисляет произведение двух мнгочленов от двух переменных
    :param a:
    :param b:
    :param size: размер по которому я 'обрезаю' многочлен.
                 Нужно для того, чтобы все степени многочлена
                 по х не превосходили заданного значение
                 (оно как раз и задаётся через size)
    :return:
    """
    ans = [[0]] * (len(a) + len(b) - 1)
    for i in range(len(a)):
        # хотим вычислить коэффициент при x^((len(a) + len(b) - 1) - i)
        for j in range(len(b)):
            if (len(a) + len(b) - 1) - x_size <= i + j:
                ans[i + j] = add_y(ans[i + j], mult_y(a[i], b[j], y_pow))
    i = 0
    while i < len(ans) and ans[i] == [0]:
        i += 1
    if i == len(ans):
        return [[0]]
    ans = ans[i:]
    return ans[-x_size:]


def make_equation(arr, xy_equation=True):
    a, b, c, d, x_size, y_pow = arr
    x = [[1], [0]]
    y = [[1, 0]]
    minus_one = [[-1]]
    if (xy_equation):
        return mult(x,
                    add(mult(a, b, 2 * x_size + 1, y_pow),
                        mult(mult(add(y, minus_one),
                                  c,
                                  2 * x_size + 1,
                                  y_pow),
                             d,
                             2 * x_size + 1,
                             y_pow)),
                    2 * x_size + 1, 
                    y_pow)[-2 * x_size + 1:]
    else:
        return mult(x,
                    add(mult(a, b, 2 * x_size + 1, y_pow),
                        mult(mult(minus_one,
                                  c,
                                  2 * x_size + 1,
                                  y_pow),
                             d,
                             2 * x_size + 1,
                             y_pow)),
                    2 * x_size + 1,
                    y_pow)[-2 * x_size + 1:]

In [ ]:
make_equation([1, 3, 4, 2, 2, 1])

In [3]:
def equations_to_series(equations: list, n: int, y_pow: int, xy_equation=True):
    """
    метод принимает на вход массив уравнений одной системы и желаемую длину ряда
    метод возвращает разложение функции F(x, y) в ряд по x
    """
#     from my_poly_functions import add, make_equation

    x = [[1], [0]]

    variable = {}
    f = add(x, [[0]])
    variable[0] = f

    for i in range(len(equations)):
        variable[equations[i][0]] = [[0]]

    for i in range(1, n - 1):
        arguments = []
        for j in range(1, len(variable)):
            arguments.append([variable[equations[j - 1][1]],
                              variable[equations[j - 1][2]],
                              variable[equations[j - 1][3]],
                              variable[equations[j - 1][4]], n, y_pow])
#         result = parallel_setup.pool.map(make_equation, arguments)
        result = pool.map(make_equation, arguments)


        for j in range(len(variable) - 1):
            variable[j + 1] = result[j]

        f = add(x, variable[1])
        variable[0] = f

    new_a = make_equation([variable[equations[0][1]], variable[equations[0][2]],
                           variable[equations[0][3]], variable[equations[0][4]], n + 1, y_pow],
                          xy_equation)
    a = new_a
    f = add(x, a)
    cut_f = f[(-2 * n):]
    return cut_f

In [4]:
from multiprocessing import Pool, cpu_count
global pool
pool = Pool(cpu_count())

In [5]:
arr = [[1, 0, 0, 2, 0],
       [2, 0, 1, 2, 1]]

In [6]:
n = 13
show_global(equations_to_series(arr, (n + 1) // 2, 2) )

'(20*y^2 + 80*y^1 + 32)*x^13 + (2*y^2 + 24*y^1 + 16)*x^11 + (6*y^1 + 8)*x^9 + (y^1 + 4)*x^7 + (2)*x^5 + (1)*x^3 + (1)*x^1 '

In [ ]:
n = 
show_global(equations_to_series(arr, (n + 1) // 2), glob='y', loc='x')

In [ ]:
N = 8
# 
# 
# 
# 
x = [np.poly1d([1]), np.poly1d([0])]
y = [np.poly1d([1, 0])]
minus_one = [np.poly1d([-1])]

a = [np.poly1d([0])]
b = [np.poly1d([0])]
f = add(x, a)

variable = {}
variable[0] = f
variable[1] = a
variable[2] = b


import time
start = time.time()
for i in range(1, N):
#     new_a = mult(x, 
#                  (add(mult(add(a, x), 
#                            add(a, x)), 
#                       mult(mult(add(y, minus_one),
#                                 add(x, a)),
#                            b))))
    print(variable)
    new_a = make_equlation(f, f, f, b)
#     print(1, f, f, f, b)
#     print(new_a)
    
    print ('new_a', time.time() - start)
    start = time.time()
#     new_b = mult(x, 
#                  (add(mult(add(a, x), 
#                            a), 
#                       mult(mult(add(y, minus_one),
#                                 a),
#                            b))))
    new_b = make_equlation(f, a, a, b)
#     print(2, f, a, a, b)
#     print(new_b)
    print ('new_b', time.time() - start)
    start = time.time()
    a = new_a
    b = new_b

    f = add(x, a)
    
    variable[0] = f
    variable[1] = a
    variable[2] = b
    print(variable)
    
    cutted_f = f[(-2 * i - 2):]
#     f = add(x, cutted)
    print(show(cutted_f), end='\n\n')
    
new_a = mult(x, 
             (add(mult(f, 
                       f), 
                  mult(mult(add(y, minus_one),
                           f),
                       b))))
print ('new_a', time.time() - start)
a = new_a
f = add(x, a)
cutted_f = f[(-2 * i - 4):]
print(show(cutted_f), end='\n\n')
# #     cutted
# #     print(a)
#     cutted = a[(-2 * i - 2):]
# #     print(cutted)
#     f = add(x, cutted)
#     print(show(f), end='\n\n')
    

In [ ]:
N = 8
# 
# 
# 
# 
x = [np.poly1d([1]), np.poly1d([0])]
y = [np.poly1d([1, 0])]
minus_one = [np.poly1d([-1])]

a = [np.poly1d([0])]
b = [np.poly1d([0])]
c = [np.poly1d([0])]
d = [np.poly1d([0])]
e = [np.poly1d([0])]
f = add(x, a)

import time
start = time.time()
null = start
for i in range(1, N - 1):
    new_a = make_equlation(f, f, b, c)
    print ('new_a', time.time() - start)
    
    start = time.time()
    new_b = make_equlation(f, a, b, c)
    print ('new_b', time.time() - start)
    
    start = time.time()
    new_c = make_equlation(f, a, e, c)
    print ('new_c', time.time() - start)
    
    start = time.time()
    new_d = make_equlation(f, a, b, c)
    print ('new_d', time.time() - start)
    
    start = time.time()
    new_e = make_equlation(a, a, b, c)
    print ('new_e', time.time() - start)
    
    a = new_a
    b = new_b
    c = new_c
    d = new_d
    e = new_e
    f = add(x, a)
    cutted_f = f[(-2 * i - 2):]
    print(show(cutted_f), end='\n\n')

start = time.time()
new_a = make_equlation(f, f, b, c)
print ('new_a', time.time() - start)
a = new_a
f = add(x, a)
cutted_f = f[(-2 * i - 4):]
print(show(cutted_f), end='\n\n')
print('всего заняло:', time.time() - null)

In [ ]:
new_a = make_equlation(f, f, b, c)
print ('new_a', time.time() - start)
a = new_a
f = add(x, a)
cutted_f = f[-15:]
print(show(cutted_f), end='\n\n')

In [ ]:
def make_equlation(a, b, c, d):
    return eval('mult(x, (add(mult(a, b), mult(mult(add(y, minus_one), c),d))))')

In [ ]:
def make_equlation(a, b, c, d, xy_equation=True):
    x = [np.poly1d([int(1)]), np.poly1d([int(0)])]
    y = [np.poly1d([int(1), int(0)])]
    minus_one = [np.poly1d([int(-1)])]
    if (xy_equation):
        return mult(x,
                    add(mult(a, b),
                        mult(mult(add(y, minus_one),
                                  c),
                             d)))
    else:
        return mult(x,
                    add(mult(a, b),
                        mult(mult(minus_one,
                                  c),
                             d)))

In [ ]:
# import time
# start = time.time()
k = 21
N = int((k + 1) / 2)
# 
# 
# 
# 
# np.dtype('O')
x = [np.poly1d([int(1)]), np.poly1d([int(0)])]
y = [np.poly1d([int(1), int(0)])]
minus_one = [np.poly1d([int(-1)])]


# arr = [[1, 0, 0, 2, 0],
#        [2, 3, 0, 2, 0],
#        [3, 1, 0, 2, 0]]

arr = [[1, 0, 0, 0, 2],
       [2, 0, 1, 1, 2]]

arr = [[1, 0, 0, 2, 0],
       [2, 3, 0, 2, 0],
       [3, 4, 0, 2, 0],
       [4, 5, 0, 2, 0],
       [5, 1, 0, 2, 0]]

variable = {}
f = add(x, [np.poly1d([int(0)])])
variable[0] = f

for i in range(len(arr)):
    variable[arr[i][0]] = [np.poly1d([int(0)])]


import time
start = time.time()
null = start
for i in range(1, N - 1):
#     print('>>>>>>>>>>')
#     print(variable)
    new_variable = {}
    for j in range(1, len(variable)):
#         print(j, variable[arr[j - 1][1]], variable[arr[j - 1][2]], 
#                  variable[arr[j - 1][3]], variable[arr[j - 1][4]])
        new_variable[j] = make_equlation(variable[arr[j - 1][1]], variable[arr[j - 1][2]], 
                                         variable[arr[j - 1][3]], variable[arr[j - 1][4]])
        new_variable[j] = new_variable[j][-2*N + 1:]
#         print(new_variable[j])
#     print(new_variable)
    for j in range(1, len(variable)):
        variable[j] = new_variable[j]
    
    f = add(x, variable[1])
    variable[0] = f
    cutted_f = f[(-2 * i - 2):]
    print(show(cutted_f), end='\n\n')

start = time.time()
new_a = make_equlation(variable[arr[0][1]], variable[arr[0][2]], 
                       variable[arr[0][3]], variable[arr[0][4]])
print ('new_a', time.time() - start)
a = new_a
f = add(x, a)
# print(i)
# print(f)
сutted_f = f
# print(show(cutted_f), end='\n\n')
print(show(f[-N* 2:]))
print('всего заняло:', time.time() - null)

In [ ]:
(y^5 + 10*y^4 + 55*y^3 + 220*y^2 + 715*y^1 + 15795.0)*x^21 + (y^4 + 9*y^3 + 45*y^2 + 165*y^1 + 4642.0)*x^19 + (y^3 + 8*y^2 + 36*y^1 + 1385.0)*x^17 + (y^2 + 7*y^1 + 421.0)*x^15 + (y^1 + 131.0)*x^13 + (42.0)*x^11 + (14.0)*x^9 + (5.0)*x^7 + (2.0)*x^5 + (1.0)*x^3 + (1.0)*x^1 

In [ ]:
type(x[0][3])

In [ ]:
# 5
1 0 0 2 0
2 3 0 4 0
3 0 5 2 5
4 3 5 4 5
5 1 0 2 0

In [ ]:
13    1.4490985870361328     0.3978149890899658
15    4.538865804672241      0.6532938480377197
17    19.676887273788452     1.096303939819336
21    331.8076431751251      


7 13   107.94368362426758
       36.235427379608154
    
заняло  0.0 часов 7.0 минут 41.352500438690186 секунд

In [ ]:
# np.dtype('O')
a = np.poly1d([1, 4, 2, 5], dtype=dt)
b = np.poly1d([ 1, 0, 4])
type(b[1])

In [ ]:
np.poly1d([1, 4, 2, 5], type = 'O')


In [ ]:
dt = np.dtype('O')
(np.array([3, 5, 3])).astype(int)
type((np.poly1d([3, 5, 3])).astype(int)[1])


In [ ]:
np.array([3, 5])

In [ ]:
x = np.array([3, 5], dtype=dt)

In [ ]:
u